# Install Require System

In [1]:
!sudo apt update
!sudo apt install -y libmagic-dev
%pip install imbalanced-learn

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease  
Get:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]33m
Fetched 336 kB in 3s (116 kB/s)   
Reading package lists... Done
Building dependency tree       
Reading state information... Done
44 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libmagic-dev is already the newest version (1:5.38-4).
0 upgraded, 0 newly installed, 0 to remove and 44 not upgraded.
Note: you may need to restart the kernel to use updated packages.


# Crate Directory and Output File

In [2]:
#Use '/resource' to syncronize folder with host

!mkdir -p ~/output/labeled-data/multi

dataset_path = "/home/jovyan/Dataset"
output_path  = "/home/jovyan/output/labeled-data/multi"

# Import Package

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

rus = RandomUnderSampler(random_state=0)
ros = RandomOverSampler(random_state=0)

# Data

In [4]:
train_data = pd.read_csv(dataset_path+'/Botnet-Training.csv')
test_data = pd.read_csv(dataset_path+'/Botnet-Testing.csv')

test_data.head(3)

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,8.6.0.1-8.0.6.4-0-0-0,8.6.0.1,0,8.0.6.4,0,0,01/01/1970 07:00:06 AM,61804233,13,0,...,0,910704.5,280640.074851,1109147.0,712262.0,8.715683e+06,3.826949e+06,1.512089e+07,5.587976e+06,NeedManualLabel
1,147.32.84.180-147.32.84.171-1040-139-6,147.32.84.180,1040,147.32.84.171,139,6,11/08/2011 03:33:48 PM,14119,10,10,...,20,0.0,0.000000,0.0,0.0,1.313052e+15,0.000000e+00,1.313052e+15,1.313052e+15,NeedManualLabel
2,147.32.84.180-147.32.84.19-1041-139-6,147.32.84.180,1041,147.32.84.19,139,6,11/08/2011 03:33:48 PM,40800,4,4,...,20,0.0,0.000000,0.0,0.0,1.313052e+15,0.000000e+00,1.313052e+15,1.313052e+15,NeedManualLabel


In [5]:
train_data.head(3)

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,8.6.0.1-8.0.6.4-0-0-0,8.6.0.1,0,8.0.6.4,0,0,10/08/2011 04:01:40 PM,59086131,8,0,...,0,2987276.0,0.0,2987276.0,2987276.0,3.282417e+14,6.564834e+14,1.312967e+15,5.999291e+06,NeedManualLabel
1,147.32.84.180-147.32.84.255-137-137-17,147.32.84.180,137,147.32.84.255,137,17,10/08/2011 04:04:23 PM,12452268,38,0,...,8,0.0,0.0,0.0,0.0,1.312967e+15,0.000000e+00,1.312967e+15,1.312967e+15,NeedManualLabel
2,147.32.84.180-147.32.80.9-1025-53-17,147.32.84.180,1025,147.32.80.9,53,17,10/08/2011 04:04:24 PM,118741070,6,3,...,8,2276383.0,0.0,2276383.0,2276383.0,6.564836e+14,9.284078e+14,1.312967e+15,1.161281e+08,NeedManualLabel


## Handling Missing Drop

In [6]:
train_data.replace([np.inf, -np.inf], np.nan, inplace=True)
test_data.replace([np.inf, -np.inf], np.nan, inplace=True)

train_data.dropna(inplace=True)
test_data.dropna(inplace=True)

In [7]:
# Train Data

#Neris
train_data.loc[(train_data['Src IP']=='147.32.84.180')| 
               (train_data['Dst IP']=='147.32.84.180'), 'Label'] = 1
#Rbot
train_data.loc[(train_data['Src IP']=='147.32.84.170') | 
               (train_data['Dst IP']=='147.32.84.170'), 'Label'] = 2
#virut
train_data.loc[(train_data['Src IP']=='147.32.84.160')| 
               (train_data['Dst IP']=='147.32.84.160'), 'Label'] = 3
#irc
train_data.loc[(train_data['Src IP']=='192.168.5.122') & (train_data['Dst IP']=='198.164.30.2'), 'Label'] = 4
train_data.loc[(train_data['Src IP']=='192.168.2.110') & (train_data['Dst IP']=='192.168.5.122'), 'Label'] = 4
train_data.loc[(train_data['Src IP']=='192.168.4.118') & (train_data['Dst IP']=='192.168.5.122'), 'Label'] = 4
train_data.loc[(train_data['Src IP']=='192.168.2.113') & (train_data['Dst IP']=='192.168.5.122'), 'Label'] = 4
train_data.loc[(train_data['Src IP']=='192.168.1.103') & (train_data['Dst IP']=='192.168.5.122'), 'Label'] = 4
train_data.loc[(train_data['Src IP']=='192.168.4.120') & (train_data['Dst IP']=='192.168.5.122'), 'Label'] = 4
train_data.loc[(train_data['Src IP']=='192.168.2.112') & (train_data['Dst IP']=='192.168.2.113'), 'Label'] = 4
train_data.loc[(train_data['Src IP']=='192.168.1.105') & (train_data['Dst IP']=='192.168.5.122'), 'Label'] = 4
#normal
train_data.loc[(train_data['Label']=='NeedManualLabel'), 'Label'] = 0

In [8]:
# Test Data

#Neris
test_data.loc[(test_data['Src IP']=='147.32.84.180')| 
               (test_data['Dst IP']=='147.32.84.180'), 'Label'] = 1
#Rbot
test_data.loc[(test_data['Src IP']=='147.32.84.170') | 
               (test_data['Dst IP']=='147.32.84.170'), 'Label'] = 2
#virut
test_data.loc[(test_data['Src IP']=='147.32.84.160')| 
               (test_data['Dst IP']=='147.32.84.160'), 'Label'] = 3
#irc
test_data.loc[(test_data['Src IP']=='192.168.5.122') & (test_data['Dst IP']=='198.164.30.2'), 'Label'] = 4
test_data.loc[(test_data['Src IP']=='192.168.2.110') & (test_data['Dst IP']=='192.168.5.122'), 'Label'] = 4
test_data.loc[(test_data['Src IP']=='192.168.4.118') & (test_data['Dst IP']=='192.168.5.122'), 'Label'] = 4
test_data.loc[(test_data['Src IP']=='192.168.2.113') & (test_data['Dst IP']=='192.168.5.122'), 'Label'] = 4
test_data.loc[(test_data['Src IP']=='192.168.1.103') & (test_data['Dst IP']=='192.168.5.122'), 'Label'] = 4
test_data.loc[(test_data['Src IP']=='192.168.4.120') & (test_data['Dst IP']=='192.168.5.122'), 'Label'] = 4
test_data.loc[(test_data['Src IP']=='192.168.2.112') & (test_data['Dst IP']=='192.168.2.113'), 'Label'] = 4
test_data.loc[(test_data['Src IP']=='192.168.1.105') & (test_data['Dst IP']=='192.168.5.122'), 'Label'] = 4
#menti
test_data.loc[(test_data['Src IP']=='147.32.84.150') | (test_data['Dst IP']=='147.32.84.150'), 'Label'] = 5
#sogou
test_data.loc[(test_data['Src IP']=='147.32.84.140') | (test_data['Dst IP']=='147.32.84.140'), 'Label'] = 6
#murlo
test_data.loc[(test_data['Src IP']=='147.32.84.130') | (test_data['Dst IP']=='147.32.84.130'), 'Label'] = 7
#blackhole
test_data.loc[(test_data['Src IP']=='10.0.2.15') | (test_data['Dst IP']=='10.0.2.15'), 'Label'] = 8
test_data.loc[(test_data['Src IP']=='192.168.106.141') | (test_data['Dst IP']=='192.168.106.141'), 'Label'] = 8
test_data.loc[(test_data['Src IP']=='192.168.106.131') | (test_data['Dst IP']=='192.168.106.131'), 'Label'] = 8
#tbot
test_data.loc[(test_data['Src IP']=='172.16.253.130') | (test_data['Dst IP']=='172.16.253.130'), 'Label'] = 9
test_data.loc[(test_data['Src IP']=='172.16.253.131') | (test_data['Dst IP']=='172.16.253.131'), 'Label'] = 9
test_data.loc[(test_data['Src IP']=='172.16.253.129') | (test_data['Dst IP']=='172.16.253.129'), 'Label'] = 9
test_data.loc[(test_data['Src IP']=='172.16.253.240') | (test_data['Dst IP']=='172.16.253.240'), 'Label'] = 9
#weasel
test_data.loc[(test_data['Src IP']=='74.78.117.238') | (test_data['Dst IP']=='74.78.117.238'), 'Label'] = 10
test_data.loc[(test_data['Src IP']=='158.65.110.24') | (test_data['Dst IP']=='158.65.110.24'), 'Label'] = 10
#zeus
test_data.loc[(test_data['Src IP']=='192.168.3.35') | (test_data['Dst IP']=='192.168.3.35'), 'Label'] = 11
test_data.loc[(test_data['Src IP']=='192.168.3.25') | (test_data['Dst IP']=='192.168.3.25'), 'Label'] = 11
test_data.loc[(test_data['Src IP']=='192.168.3.65') | (test_data['Dst IP']=='192.168.3.65'), 'Label'] = 11
test_data.loc[(test_data['Src IP']=='172.29.0.116') | (test_data['Dst IP']=='172.29.0.116'), 'Label'] = 11
#zero access
test_data.loc[(test_data['Src IP']=='172.16.253.132') | (test_data['Dst IP']=='172.16.253.132'), 'Label'] = 12
test_data.loc[(test_data['Src IP']=='192.168.248.165') | (test_data['Dst IP']=='192.168.248.165'), 'Label'] = 12
#trojan
test_data.loc[(test_data['Src IP']=='172.29.0.109') | (test_data['Dst IP']=='172.29.0.109'), 'Label'] = 13
#smoke
test_data.loc[(test_data['Src IP']=='10.37.130.4') | (test_data['Dst IP']=='10.37.130.4'), 'Label'] = 14
#normal
test_data.loc[(test_data['Label']=='NeedManualLabel'), 'Label'] = 0

In [9]:
train_data['Label'].value_counts()

0    247089
2     45342
4     39485
1     22492
3      1748
Name: Label, dtype: int64

In [10]:
test_data['Label'].value_counts()

0     127891
10     85502
3      42725
1      26663
7      12946
5       4970
4       4105
12      1828
9        967
8        747
11       424
2        226
14        84
6         82
13        27
Name: Label, dtype: int64

In [11]:
values = [0, 1, 2, 3, 4]

test_data = test_data[test_data.Label.isin(values) == False]

# Split Data by Value

In [12]:
median = list(train_data['Label'].value_counts())[-1]

# OverValue & UnderValue
label_ov = list(test_data['Label'].value_counts().loc[lambda x: x>=median].index)
label_uv = list(test_data['Label'].value_counts()
                .loc[lambda x: x.between(median/4, median)].index)

In [13]:
test_ov = test_data[test_data.Label.isin(label_ov) == True] #OverValue
test_uv = test_data[test_data.Label.isin(label_uv) == True] #UderValue

In [14]:
test_ov['Label'].value_counts()

10    85502
7     12946
5      4970
12     1828
Name: Label, dtype: int64

In [15]:
test_uv['Label'].value_counts()

9    967
8    747
Name: Label, dtype: int64

## UnderSampling

In [16]:
data_ov = pd.concat([train_data, test_ov]) #OverValue

In [17]:
col = data_ov.drop(['Label'], axis=1)
tar = data_ov[['Label']]

In [18]:
col.head(4)

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,8.6.0.1-8.0.6.4-0-0-0,8.6.0.1,0,8.0.6.4,0,0,10/08/2011 04:01:40 PM,59086131,8,0,...,0,0,2987276.0,0.0,2987276.0,2987276.0,3.282417e+14,6.564834e+14,1.312967e+15,5.999291e+06
1,147.32.84.180-147.32.84.255-137-137-17,147.32.84.180,137,147.32.84.255,137,17,10/08/2011 04:04:23 PM,12452268,38,0,...,37,8,0.0,0.0,0.0,0.0,1.312967e+15,0.000000e+00,1.312967e+15,1.312967e+15
2,147.32.84.180-147.32.80.9-1025-53-17,147.32.84.180,1025,147.32.80.9,53,17,10/08/2011 04:04:24 PM,118741070,6,3,...,5,8,2276383.0,0.0,2276383.0,2276383.0,6.564836e+14,9.284078e+14,1.312967e+15,1.161281e+08
3,147.32.84.180-94.63.149.152-1040-80-6,147.32.84.180,1040,94.63.149.152,80,6,10/08/2011 04:06:36 PM,180643,26,10,...,2,20,0.0,0.0,0.0,0.0,1.312967e+15,0.000000e+00,1.312967e+15,1.312967e+15


In [19]:
print(col.shape)
print(tar.shape)

(461402, 83)
(461402, 1)


In [20]:
tar=tar.astype('int')
tar.dtypes

Label    int64
dtype: object

In [21]:
#UnderSampling
X_res, Y_res = rus.fit_resample(col, tar)

In [22]:
dataResampling = pd.concat([X_res, Y_res],axis=1)
dataResampling.head(5)

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,203.73.24.75-192.168.2.109-80-2121-6,203.73.24.75,80,192.168.2.109,2121,6,12/06/2010 09:33:30 PM,824776,5,2,...,20,0.0,0.0,0.0,0.0,1.276353e+15,0.000000e+00,1.276353e+15,1.276353e+15,0
1,192.168.2.111-66.179.20.163-3655-80-6,192.168.2.111,3655,66.179.20.163,80,6,12/06/2010 11:05:38 PM,49287464,3,0,...,20,0.0,0.0,0.0,0.0,6.381794e+14,9.025219e+14,1.276359e+15,4.928731e+07,0
2,203.73.24.75-192.168.2.109-80-3828-6,203.73.24.75,80,192.168.2.109,3828,6,13/06/2010 07:05:34 AM,2922422,42,22,...,20,0.0,0.0,0.0,0.0,1.276388e+15,0.000000e+00,1.276388e+15,1.276388e+15,0
3,192.168.4.121-192.168.5.122-51526-143-6,192.168.4.121,51526,192.168.5.122,143,6,12/06/2010 11:19:52 AM,31006,14,26,...,20,0.0,0.0,0.0,0.0,1.276316e+15,0.000000e+00,1.276316e+15,1.276316e+15,0
4,172.16.2.13-82.255.155.203-2069-6047-6,172.16.2.13,2069,82.255.155.203,6047,6,08/10/2007 09:25:41 PM,44055886,11,10,...,20,1623322.0,0.0,1623322.0,1623322.0,5.959268e+14,8.427677e+14,1.191854e+15,4.243256e+07,0


In [23]:
print("Before UnderSampling: ", tar.value_counts())
print("After UnderSampling: ", Y_res.value_counts())
print(dataResampling.shape)

Before UnderSampling:  Label
0        247089
10        85502
2         45342
4         39485
1         22492
7         12946
5          4970
12         1828
3          1748
dtype: int64
After UnderSampling:  Label
0        1748
1        1748
2        1748
3        1748
4        1748
5        1748
7        1748
10       1748
12       1748
dtype: int64
(15732, 84)


# OverSampling

In [24]:
data_uv = pd.concat([dataResampling, test_uv])

In [25]:
data_uv['Label'].value_counts()

0     1748
1     1748
2     1748
3     1748
4     1748
5     1748
7     1748
10    1748
12    1748
9      967
8      747
Name: Label, dtype: int64

In [26]:
col = data_uv.drop(['Label'], axis=1)
tar = data_uv[['Label']]

In [27]:
col.head(4)

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,203.73.24.75-192.168.2.109-80-2121-6,203.73.24.75,80,192.168.2.109,2121,6,12/06/2010 09:33:30 PM,824776,5,2,...,1,20,0.0,0.0,0.0,0.0,1.276353e+15,0.000000e+00,1.276353e+15,1.276353e+15
1,192.168.2.111-66.179.20.163-3655-80-6,192.168.2.111,3655,66.179.20.163,80,6,12/06/2010 11:05:38 PM,49287464,3,0,...,1,20,0.0,0.0,0.0,0.0,6.381794e+14,9.025219e+14,1.276359e+15,4.928731e+07
2,203.73.24.75-192.168.2.109-80-3828-6,203.73.24.75,80,192.168.2.109,3828,6,13/06/2010 07:05:34 AM,2922422,42,22,...,39,20,0.0,0.0,0.0,0.0,1.276388e+15,0.000000e+00,1.276388e+15,1.276388e+15
3,192.168.4.121-192.168.5.122-51526-143-6,192.168.4.121,51526,192.168.5.122,143,6,12/06/2010 11:19:52 AM,31006,14,26,...,11,20,0.0,0.0,0.0,0.0,1.276316e+15,0.000000e+00,1.276316e+15,1.276316e+15


In [28]:
print(col.shape)
print(tar.shape)

(17446, 83)
(17446, 1)


In [29]:
tar=tar.astype('int')
tar.dtypes

Label    int64
dtype: object

In [30]:
#OverSampling
X_res, Y_res = ros.fit_resample(col, tar)

In [31]:
dataResampling = pd.concat([X_res, Y_res],axis=1)
dataResampling.head(5)

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,203.73.24.75-192.168.2.109-80-2121-6,203.73.24.75,80,192.168.2.109,2121,6,12/06/2010 09:33:30 PM,824776,5,2,...,20,0.0,0.0,0.0,0.0,1.276353e+15,0.000000e+00,1.276353e+15,1.276353e+15,0
1,192.168.2.111-66.179.20.163-3655-80-6,192.168.2.111,3655,66.179.20.163,80,6,12/06/2010 11:05:38 PM,49287464,3,0,...,20,0.0,0.0,0.0,0.0,6.381794e+14,9.025219e+14,1.276359e+15,4.928731e+07,0
2,203.73.24.75-192.168.2.109-80-3828-6,203.73.24.75,80,192.168.2.109,3828,6,13/06/2010 07:05:34 AM,2922422,42,22,...,20,0.0,0.0,0.0,0.0,1.276388e+15,0.000000e+00,1.276388e+15,1.276388e+15,0
3,192.168.4.121-192.168.5.122-51526-143-6,192.168.4.121,51526,192.168.5.122,143,6,12/06/2010 11:19:52 AM,31006,14,26,...,20,0.0,0.0,0.0,0.0,1.276316e+15,0.000000e+00,1.276316e+15,1.276316e+15,0
4,172.16.2.13-82.255.155.203-2069-6047-6,172.16.2.13,2069,82.255.155.203,6047,6,08/10/2007 09:25:41 PM,44055886,11,10,...,20,1623322.0,0.0,1623322.0,1623322.0,5.959268e+14,8.427677e+14,1.191854e+15,4.243256e+07,0


In [32]:
print("Before OverSampling: ", tar.value_counts())
print("After OverSampling: ", Y_res.value_counts())
print(dataResampling.shape)

Before OverSampling:  Label
0        1748
1        1748
2        1748
3        1748
4        1748
5        1748
7        1748
10       1748
12       1748
9         967
8         747
dtype: int64
After OverSampling:  Label
0        1748
1        1748
2        1748
3        1748
4        1748
5        1748
7        1748
8        1748
9        1748
10       1748
12       1748
dtype: int64
(19228, 84)


In [33]:
dataResampling.to_csv(output_path+'/FullDataResampling.csv')